# Import relevant libraries

In [34]:
#!pip install matplotlib pandas scikit-learn
#!pip install --upgrade pip
#!pip install statsmodels

In [35]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from math import sin, cos, sqrt, atan2, radians
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (auc, classification_report, roc_auc_score, accuracy_score,
                             f1_score, log_loss, roc_curve, confusion_matrix, precision_score, recall_score)
import statsmodels.formula.api as smf

np.random.seed(0)

#Load the Data 

In [36]:
tournament_Data = pd.read_csv('https://raw.githubusercontent.com/SethiDeepika/AIML/main/NCAA_Tourney_2002_2025.csv')
tournament_Data.head()
#tournament_TestData = pd.read_csv('https://raw.githubusercontent.com/SethiDeepika/AIML/main/NCAA_Tourney_2002_2025.csv').query("season==2019").reset_index(drop=True)
#tournament_TestData.head()

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,...,team1_adjoe,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id
0,1314,81.0,1181,77.0,N,0.0,W08,X02,8,2,...,113.035,100.974,96.9911,68.2765,67.4185,117.1520,119.3570,98.2346,95.6444,2022-1314-1181
1,1242,81.0,1437,65.0,N,0.0,Y01,Z02,1,2,...,119.388,97.191,93.9009,64.1915,62.5758,112.8450,117.9210,97.6798,93.8099,2022-1242-1437
2,1242,72.0,1314,69.0,N,0.0,Y01,W08,1,8,...,119.388,97.191,93.9009,70.2181,70.1745,109.4160,113.0350,100.9740,96.9911,2022-1242-1314
3,1242,76.0,1274,50.0,N,0.0,Y01,Y10,1,10,...,119.388,97.191,93.9009,67.5221,67.3101,110.3680,114.7570,104.9830,102.1320,2022-1242-1274
4,1314,69.0,1389,49.0,N,0.0,W08,W15,8,15,...,113.035,100.974,96.9911,67.0262,65.9923,98.7557,98.7475,92.3289,94.4745,2022-1314-1389


In [37]:
# The data has team1 as the winner. We need to create balanced training data
# by also including the reverse matchups (team2 vs team1)
print("\nPreparing training data...")



Preparing training data...


In [71]:
df_winner = tournament_Data.copy()
df_winner['target'] = 1  # team1 wins
df_loser = tournament_Data.copy()
df_loser['target'] = 0  # team2 loses

team1_cols = [c for c in tournament_Data.columns if c.startswith('team1_')]
team2_cols = [c for c in tournament_Data.columns if c.startswith('team2_')]


# Put Team 2 data into Team 1's columns
df_loser[team1_cols] = tournament_Data[team2_cols].values

# Put Team 1 data into Team 2's columns
df_loser[team2_cols] = tournament_Data[team1_cols].values

# Feature engineering
stat_cols_team1 = [
    'team1_fg2pct', 'team1_fg3pct', 'team1_ftpct', 'team1_blockpct',
    'team1_oppfg2pct', 'team1_oppfg3pct', 'team1_arate', 'team1_opparate',
    'team1_stlrate', 'team1_oppstlrate', 'team1_adjoe', 'team1_adjde'
]

stat_cols_team2 = [
    'team2_fg2pct', 'team2_fg3pct', 'team2_ftpct', 'team2_blockpct',
    'team2_oppfg2pct', 'team2_oppfg3pct', 'team2_arate', 'team2_opparate',
    'team2_stlrate', 'team2_oppstlrate', 'team2_adjoe', 'team2_adjde'
]

df_combined = pd.concat([df_winner, df_loser], axis=0).reset_index(drop=True)
df_combined = df_combined.sort_values(by=['game_id']).reset_index(drop=True)
feature_names = []

for t1, t2 in zip(stat_cols_team1, stat_cols_team2):
    base_name = t1.replace('team1_', '')
    df_combined[f'diff_{base_name}'] = df_combined[t1] - df_combined[t2]
    feature_names.append(f'diff_{base_name}')

# Seed difference
df_combined['seed_diff'] = df_combined['team2_seed'] - df_combined['team1_seed']
feature_names.append('seed_diff')

# Net efficiency difference
df_combined['team1_net_eff'] = df_combined['team1_adjoe'] - df_combined['team1_adjde']
df_combined['team2_net_eff'] = df_combined['team2_adjoe'] - df_combined['team2_adjde']
df_combined['diff_net_eff'] = df_combined['team1_net_eff'] - df_combined['team2_net_eff']
feature_names.append('diff_net_eff')



df_combined.columns



Index(['team1_id', 'team1_score', 'team2_id', 'team2_score', 'WLoc', 'num_ot',
       'team1_position', 'team2_position', 'team1_seed', 'team2_seed',
       'strongseed', 'weakseed', 'team1_region', 'team2_region', 'slot',
       'team1_teamname', 'team2_teamname', 'season', 'host', 'host_lat',
       'host_long', 'team1_lat', 'team1_long', 'team2_lat', 'team2_long',
       'team1_fg2pct', 'team1_fg3pct', 'team1_ftpct', 'team1_blockpct',
       'team1_oppfg2pct', 'team1_oppfg3pct', 'team1_oppftpct',
       'team1_oppblockpct', 'team1_f3grate', 'team1_oppf3grate', 'team1_arate',
       'team1_opparate', 'team1_stlrate', 'team1_oppstlrate', 'team2_fg2pct',
       'team2_fg3pct', 'team2_ftpct', 'team2_blockpct', 'team2_oppfg2pct',
       'team2_oppfg3pct', 'team2_oppftpct', 'team2_oppblockpct',
       'team2_f3grate', 'team2_oppf3grate', 'team2_arate', 'team2_opparate',
       'team2_stlrate', 'team2_oppstlrate', 'team1_tempo', 'team1_adjtempo',
       'team1_oe', 'team1_adjoe', 'team1_de

In [52]:
#Feature Engineering Using Claude

# ===== 1. TEMPO FEATURES =====
df_combined['diff_tempo'] = df_combined['team1_tempo'] - df_combined['team2_tempo']
df_combined['diff_adjtempo'] = df_combined['team1_adjtempo'] - df_combined['team2_adjtempo']
feature_names.extend(['diff_tempo', 'diff_adjtempo'])

# ===== 2. UPSET INDICATORS =====
df_combined['upset_potential'] = ((df_combined['seed_diff'] < 0) & (df_combined['diff_net_eff'] > 0)).astype(int)
df_combined['seed_diff_sq'] = df_combined['seed_diff'] ** 2
feature_names.extend(['upset_potential', 'seed_diff_sq'])

# ===== 3. SHOOTING EFFICIENCY =====
df_combined['team1_overall_fg'] = df_combined['team1_fg2pct'] * 0.6 + df_combined['team1_fg3pct'] * 0.4
df_combined['team2_overall_fg'] = df_combined['team2_fg2pct'] * 0.6 + df_combined['team2_fg3pct'] * 0.4
df_combined['diff_overall_fg'] = df_combined['team1_overall_fg'] - df_combined['team2_overall_fg']
feature_names.append('diff_overall_fg')

# ===== 4. DEFENSIVE PRESSURE =====
df_combined['team1_def_pressure'] = df_combined['team1_stlrate'] + df_combined['team1_blockpct']
df_combined['team2_def_pressure'] = df_combined['team2_stlrate'] + df_combined['team2_blockpct']
df_combined['diff_def_pressure'] = df_combined['team1_def_pressure'] - df_combined['team2_def_pressure']
feature_names.append('diff_def_pressure')

# ===== 5. TURNOVER MARGIN =====
df_combined['team1_to_margin'] = df_combined['team1_stlrate'] - df_combined['team1_oppstlrate']
df_combined['team2_to_margin'] = df_combined['team2_stlrate'] - df_combined['team2_oppstlrate']
df_combined['diff_to_margin'] = df_combined['team1_to_margin'] - df_combined['team2_to_margin']
feature_names.append('diff_to_margin')

# ===== 6. LUCK (EFF VS SEED) =====
seed_to_eff = {1:28, 2:24, 3:21, 4:18, 5:16, 6:14, 7:12, 8:10, 9:8, 10:6, 11:5, 12:4, 13:2, 14:0, 15:-2, 16:-5}
df_combined['team1_eff_vs_seed'] = df_combined['team1_net_eff'] - df_combined['team1_seed'].map(seed_to_eff)
df_combined['team2_eff_vs_seed'] = df_combined['team2_net_eff'] - df_combined['team2_seed'].map(seed_to_eff)
df_combined['diff_eff_vs_seed'] = df_combined['team1_eff_vs_seed'] - df_combined['team2_eff_vs_seed']
feature_names.append('diff_eff_vs_seed')

# ===== 7. MATCHUP ADVANTAGE =====
df_combined['matchup_advantage'] = (df_combined['team1_adjoe'] - df_combined['team2_adjde']) - (df_combined['team2_adjoe'] - df_combined['team1_adjde'])
feature_names.append('matchup_advantage')

# ===== 8. TEMPO MISMATCH =====
df_combined['tempo_mismatch'] = abs(df_combined['team1_tempo'] - df_combined['team2_tempo'])
feature_names.append('tempo_mismatch')

df_combined

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,...,team2_def_pressure,diff_def_pressure,team1_to_margin,team2_to_margin,diff_to_margin,team1_eff_vs_seed,team2_eff_vs_seed,diff_eff_vs_seed,matchup_advantage,tempo_mismatch
2224,1385,65.0,1113,74.0,N,0.0,X11b,X11a,11,11,...,9.783595,0.3734,0.04969,-0.001707,0.051397,3.738,6.2016,-2.4636,2.2556,0.2175
2225,1113,74.0,1385,65.0,N,0.0,X11a,X11b,11,11,...,10.156995,-0.3734,-0.001707,0.04969,-0.051397,6.2016,3.738,2.4636,-2.2556,0.2175
2226,1242,75.0,1120,89.0,N,0.0,Y04,Y05,4,5,...,16.231007,-5.151477,0.005352,0.044998,-0.039646,2.7496,6.1124,-3.3628,-11.4332,1.7759
2227,1120,89.0,1242,75.0,N,0.0,Y05,Y04,5,4,...,11.07953,5.151477,0.044998,0.005352,0.039646,6.1124,2.7496,3.3628,11.4332,1.7759
2228,1120,77.0,1246,71.0,N,1.0,Y05,Y02,5,2,...,14.289163,1.941845,0.044998,0.005336,0.039662,6.1124,3.4773,2.6351,7.2149,0.9773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,1387,52.0,1439,66.0,N,0.0,W13,W04,13,4,...,8.829087,3.021052,0.016215,0.024583,-0.008368,4.2112,6.4676,-2.2564,-13.3216,2.7529
2354,1449,78.0,1429,61.0,N,0.0,Y09,Y08,9,8,...,12.455383,4.050967,0.042336,0.002153,0.040183,5.3326,6.6567,-1.3241,-8.7379,2.7095
2355,1429,61.0,1449,78.0,N,0.0,Y08,Y09,8,9,...,16.50635,-4.050967,0.002153,0.042336,-0.040183,6.6567,5.3326,1.3241,8.7379,2.7095
2356,1459,84.0,1371,68.0,N,0.0,Y07,Y10,7,10,...,11.267574,-2.253335,0.007764,0.016671,-0.008907,8.7201,6.8974,1.8227,10.3053,1.9621


In [40]:


df_training = pd.concat([df_winner, df_loser], axis=0).reset_index(drop=True)
df_training = df_training.sort_values(by=['game_id']).reset_index(drop=True)

#df_training=df_combined[feature_names]
df_training.head()

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,...,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id,target
0,1194,78.0,1104,86.0,N,0.0,Y15,Y02,15,2,...,98.4183,99.9263,69.8636,69.9001,108.4361,111.4954,95.2313,93.877,2002-1104-1194,0
1,1104,86.0,1194,78.0,N,0.0,Y02,Y15,2,15,...,95.2313,93.877,71.2357,71.2446,100.2897,96.8669,98.4183,99.9263,2002-1104-1194,1
2,1112,86.0,1364,81.0,N,0.0,Z03,Z14,3,14,...,104.0411,96.9262,63.2345,64.7948,105.2163,105.4534,96.0965,97.6704,2002-1112-1364,1
3,1364,81.0,1112,86.0,N,0.0,Z14,Z03,14,3,...,96.0965,97.6704,74.1462,72.8207,111.0077,117.3877,104.0411,96.9262,2002-1112-1364,0
4,1112,68.0,1461,60.0,N,0.0,Z03,Z11,3,11,...,104.0411,96.9262,69.6172,70.4124,105.3654,106.037,97.0568,96.6601,2002-1112-1461,1


In [41]:
df_training.shape

(3028, 67)

In [42]:
#!pip install tensorflow

In [43]:
# Keras imports
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Sklearn for preprocessing and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [81]:
df_combined
#df_combined[df_combined['season'] != '2019']
df_combined[df_combined['season'] != 2019]

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,...,diff_arate,diff_opparate,diff_stlrate,diff_oppstlrate,diff_adjoe,diff_adjde,seed_diff,team1_net_eff,team2_net_eff,diff_net_eff
0,1194,78.0,1104,86.0,N,0.0,Y15,Y02,15,2,...,6.0473,9.3519,0.0229,0.0195,-14.6285,6.0493,-13,-3.0594,17.6184,-20.6778
1,1104,86.0,1194,78.0,N,0.0,Y02,Y15,2,15,...,-6.0473,-9.3519,-0.0229,-0.0195,14.6285,-6.0493,13,17.6184,-3.0594,20.6778
2,1112,86.0,1364,81.0,N,0.0,Z03,Z14,3,14,...,-9.3072,-3.6991,-0.0033,0.0061,11.9343,-0.7442,11,20.4615,7.783,12.6785
3,1364,81.0,1112,86.0,N,0.0,Z14,Z03,14,3,...,9.3072,3.6991,0.0033,-0.0061,-11.9343,0.7442,-11,7.783,20.4615,-12.6785
4,1112,68.0,1461,60.0,N,0.0,Z03,Z11,3,11,...,4.585,0.9468,-0.0125,-0.0229,11.3507,0.2661,8,20.4615,9.3769,11.0846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,1417,72.0,1429,47.0,N,0.0,X07,X10,7,10,...,-0.60981,-0.522621,-0.003288,-0.01391,-3.885,-10.4223,3,21.7623,15.225,6.5373
3024,1285,66.0,1458,85.0,N,0.0,W14,W03,14,3,...,-5.538595,-3.733456,0.014429,0.013924,-11.238,14.3529,-11,0.384,25.9749,-25.5909
3025,1458,85.0,1285,66.0,N,0.0,W03,W14,3,14,...,5.538595,3.733456,-0.014429,-0.013924,11.238,-14.3529,11,25.9749,0.384,25.5909
3026,1400,80.0,1462,86.0,N,0.0,X11a,X11b,11,11,...,-16.444696,-7.771025,-0.019248,-0.00141,1.214,1.2596,0,17.1514,17.197,-0.0456


In [82]:
df_combined_Train = df_combined[df_combined['season'] != 2019]

# Features (X) and Target (y)
X = df_combined_Train[feature_names].values
y = df_combined_Train['target'].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (2894, 14)
y shape: (2894,)


In [83]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)


X_train_scaled.shape

(2894, 14)

In [84]:
model = Sequential()

# Input layer + First hidden layer
# input_dim = number of features
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Second hidden layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

# Third hidden layer
model.add(Dense(16, activation='relu'))

# Output layer (1 neuron with sigmoid for binary classification)
model.add(Dense(1, activation='sigmoid'))

print("✅ Model built!")
model.summary()

✅ Model built!


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with default learning rate
    loss='binary_crossentropy',            # Loss function for binary classification
    metrics=['accuracy']                   # Track accuracy during training
)

print("✅ Model compiled!")

✅ Model compiled!


In [86]:
early_stop = EarlyStopping(
    monitor='val_loss',      # Watch validation loss
    patience=10,             # Wait 10 epochs before stopping
    restore_best_weights=True  # Keep the best model
)

print("✅ Early stopping configured")

✅ Early stopping configured


In [87]:
print("🏋️ Training the model...\n")

history = model.fit(
    X_train_scaled, 
    y,
    epochs=100,              # Maximum epochs (early stopping will likely stop before)
    batch_size=32,           # Process 32 samples at a time
    validation_split=0.2,    # Use 20% of training data for validation
    callbacks=[early_stop],  # Use early stopping
    verbose=1                # Show progress
)

print(f"\n✅ Training complete! Stopped at epoch {len(history.history['loss'])}")

🏋️ Training the model...

Epoch 1/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6341 - loss: 0.6332 - val_accuracy: 0.6667 - val_loss: nan
Epoch 2/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7028 - loss: 0.5724 - val_accuracy: 0.6701 - val_loss: nan
Epoch 3/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6937 - loss: 0.5736 - val_accuracy: 0.6736 - val_loss: nan
Epoch 4/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7063 - loss: 0.5611 - val_accuracy: 0.6753 - val_loss: nan
Epoch 5/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7110 - loss: 0.5452 - val_accuracy: 0.6701 - val_loss: nan
Epoch 6/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7097 - loss: 0.5553 - val_accuracy: 0.6684 - val_loss: nan
Epoch 7/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7097 - loss: 0.5424 - val_accuracy: 0.6701 - val_loss: nan
Epoch 8/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7153 - loss: 0.5525 - val_accuracy: 0.668

In [88]:

df_combined_Test = df_combined[df_combined['season'] == 2019]


# Features (X) and Target (y)
X_test = df_combined_Test[feature_names].values
X_test_scaled = scaler.transform(X_test)
y_test = df_combined_Test['target'].values


test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

print("="*50)
print("📊 TEST SET RESULTS")
print("="*50)
print(f"Test Accuracy: {test_accuracy:.2%}")
print(f"Test Loss: {test_loss:.4f}")

📊 TEST SET RESULTS
Test Accuracy: 73.13%
Test Loss: 0.5262


In [ ]:
y_pred_proba = model.predict(X_test_scaled)  # Probabilities
y_pred = (y_pred_proba > 0.8).astype(int).flatten()  # Convert to 0/1

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Team2 Wins', 'Team1 Wins']))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

📋 Classification Report:
              precision    recall  f1-score   support

  Team2 Wins       0.72      0.75      0.74        67
  Team1 Wins       0.74      0.72      0.73        67

    accuracy                           0.73       134
   macro avg       0.73      0.73      0.73       134
weighted avg       0.73      0.73      0.73       134



In [ ]:
y_pred

array([[0.4467307 ],
       [0.5199011 ],
       [0.49447834],
       [0.5042174 ],
       [0.4126021 ],
       [0.67923236],
       [0.5572889 ],
       [0.3504404 ],
       [0.67280656],
       [0.40728176],
       [0.44630873],
       [0.51597816],
       [0.62672806],
       [0.33910063],
       [0.32499805],
       [0.73715657],
       [0.13854906],
       [0.97591984],
       [0.2867795 ],
       [0.8416264 ],
       [0.28191584],
       [0.705289  ],
       [0.52687687],
       [0.44857368],
       [0.35763648],
       [0.5588653 ],
       [0.3656499 ],
       [0.5940408 ],
       [0.32897854],
       [0.82606643],
       [0.214538  ],
       [0.8333043 ],
       [0.13858503],
       [0.97100693],
       [0.725802  ],
       [0.26926196],
       [0.92940074],
       [0.28097263],
       [0.73655003],
       [0.26208258],
       [0.66139936],
       [0.31882206],
       [0.75298345],
       [0.29446226],
       [0.899138  ],
       [0.20203301],
       [0.53426546],
       [0.579